<a href="https://colab.research.google.com/github/Bateyjosue/NLP_Fellowship/blob/main/igihe_scrap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Assignment
Based on the above, get the main articles from igihe from February 2022 - present

Steps to do this


1.   Get the links to the main pages from january. Create a list
2.   In each link, get all the links to the main articles
3.   For each article, get the main tag that holds the texts
4.   Get the text and store them in a txt file. The data will be used in week 2
5.   Each article its own txt file. Naming is the date_article_1


In [1]:
# Import Packages
import requests
from bs4 import BeautifulSoup

In [2]:
# MAke an API Call 
def get_content(url):
  response = requests.get(url=url)
  content = BeautifulSoup(response.content, 'lxml')
  return content

def get_data(url, month, day):
  response = requests.get(url+'{:02d}{:02d}'.format(month,day))
  return response.json()


In [3]:
data = get_data('https://archive.org/wayback/available?url=igihe.com/&timestamp=2022',1,23)


In [4]:
def check_snaps(data):
  if data['archived_snapshots']:
    time_stamp = data['timestamp'] 
    time_stamp_closest = data['archived_snapshots']['closest']['timestamp'][:8]
    result = time_stamp == time_stamp_closest
  else: 
    result = False
  return result

print(check_snaps(data))

True


In [5]:
# 1.   Get the links to the main pages from january. Create a list
def get_main_links(url):
  links = []
  for month in range(1, 11):
    for day in range(1, 32):
      data = get_data(url, month, day)
      if check_snaps(data):
        links.append(data['archived_snapshots']['closest']['url'])
  return links

main_links = get_main_links('https://archive.org/wayback/available?url=igihe.com/&timestamp=2022')
  

In [39]:
# 2.   In each link, get all the links to the main articles
def main_articles_link(links):
  article_links = []
  for link in links:
    content = get_content(link)
    for title in content.find_all('span', class_='homenews-title'):
      article_links.append(link + title.find('a')['href'])
  return article_links

main_articles = main_articles_link(main_links[:2])


In [ ]:
# 3.   For each article, get the main tag that holds the texts
def main_tag_text(main_articles):
  text = ''
  list_text = []
  for main_article in main_articles:
    response = get_content(main_article)
    main_text = response.find('div', class_='fulltext margintop10')
    if main_text != None:
      text = ''.join([i.get_text() for i in main_text.find_all('p') if i.get_text()])
      list_text.append(text)
  
  return list_text

articles_text = main_tag_text(main_articles)

In [ ]:
# 4.   Get the text and store them in a txt file. The data will be used in week 2
def get_text_from_link(url):
  articles = []
  titles = []
  for link in url:
    soup = get_content(link)
    articles.append(soup.find('div', class_='fulltext margintop10').get_text())
    with open('articles_text.txt', 'a+') as file:
      file.writelines(soup.find('div', class_='fulltext margintop10').get_text())
    # titles.append(soup.find('h3', class_='title-article'))

get_text_from_link(main_articles)

In [ ]:
# 5.   Each article its own txt file. Naming is the date_article_1
import re

link = main_articles[1]
# r = requests.get(link).content
# soup = BeautifulSoup(r, 'lxml')
# soup.find_all('div')
pattern = re.compile(r'\d+')
match = re.search(pattern, link)
date = match.group()
resp = requests.get(url= link)
soups = BeautifulSoup(resp.content, 'lxml')
div = soups.find('div', class_='fulltext margintop10')
text = ''.join([i.get_text() for i in div.find_all('p') if i.get_text()])
with open('{}_text_1.txt'.format(date), 'w+') as file:
  file.write(text)


In [30]:
content = get_content('https://www.microverse.org/')

In [32]:
testimonials = content.find_all('div', class_='orig_featured_testimonial_v2')

In [38]:
for testimonials in testimonials:
  print(testimonials.get_text())

— "For me, it was a no-brainer. I wanted to try Microverse because while the alternatives would give you a tutorial or a course, you have to do it alone, but with this, I had the accountability."“Kevin M, Senior Software Developer at Microsoft🇰🇪 Kenya
— "My life has changed 100% since Microverse. My current salary is 5.6 times more per year than my previous salary."“Gabriela C, Frontend Developer at Upstart 13🇲🇽 Mexico
— "To be successful I knew I had to develop skills in planning, discipline, and productivity. Microverse provided the structure so I could grow in these areas, which prepared me for working in the software industry."“Alex S, Full-Stack Developer at Doctolib🇩🇪 Germany
